In [596]:
# import sys
# !pip install --prefix {sys.prefix} ipywidgets

In [1]:
def initButton(desc, color, width, height):
    return ToggleButton(description = desc, disabled = False, button_style = color, 
                        layout = Layout(width = width, height = height))

def initHbox(text, maximum, step, val):
    return HBox([
        Label(text, layout = Layout(width = '22%')), 
        IntSlider(min = 0, max = maximum, step = step, value = val, layout = Layout(width = '50%'))])


def initFloatHbox(text, maximum, step, val):
    return HBox([
        Label(text, layout = Layout(width = '22%')), 
        FloatSlider(min = 0, max = maximum, step = step, value = val, layout = Layout(width = '50%'))])

#### Konfiguracja

In [2]:
def initConfPanel():
    global populationSl, bitsSl, epochSl
    confHeader = initButton('Konfiguracja', 'danger', '72%', '50px')
    populationSl = initHbox('Liczba osobników populacji', 200, 1, 20)
    bitsSl = initHbox('Dokłandność chromosomów', 20, 1, 8)
    epochSl = initHbox('Liczba epok', 500, 1, 100)

    confSliders = Box(
        children = [populationSl, bitsSl, epochSl], 
        layout = Layout(display = 'flex', flex_flow = 'column', padding = "15px")
    )
    return AppLayout(header = confHeader, center = confSliders)

#### Operatory genetyczne

In [3]:
def initGenPanel():
    global selectionBtn, crossoverBtn, mutationBtn, selectionSl, crossoverSl, mutationSl
    
    genHeader = initButton('Operatory genetyczne', 'danger', '72%', '50px')

    selectionBtn = ToggleButtons(
        options = ['najlepszych', 'kołem ruletki', 'turniejowa', 'strategia elitarna'],
        description = 'Selekcja: ', button_style = 'danger')

    crossoverBtn = ToggleButtons(
        options = ['jednopunktowe', 'dwupunktowe', 'trzypunktowe', 'jednorodne'],
        description = 'Krzyżowanie: ', button_style = 'warning')

    mutationBtn = ToggleButtons(
        options = ['jednopunktowa', 'dwupunktowa', 'brzegowa'],
        description = 'Mutacja: ', button_style = 'success')

    selectionSl = initHbox('Procent osobników', 100, 1, 30) 
    crossoverSl = initFloatHbox('Prawdopodobieństwo krzyżowania', 1, 0.1, 0.9)
    mutationSl = initFloatHbox('Prawdopodobieństwo mutacji', 1, 0.1, 0.2)

    buttons = Box(
        children = [selectionBtn, crossoverBtn, mutationBtn], 
        layout = Layout(display = 'flex', flex_flow = 'column')
    )
    geneticSliders = Box(
        children = [selectionSl, crossoverSl, mutationSl], 
        layout = Layout(display = 'flex', flex_flow = 'column', padding = "15px")
    )
    selectionBtn.observe(selectionBtnClicked, 'value')
    box = Box(children = [buttons, geneticSliders], layout = Layout(display = 'flex', flex_flow = 'column'))
    
    return AppLayout(header = genHeader, center = box)

#### Event listeners

In [4]:
def incrementEpochProgress():
    epochProgress.value += 1
    
def updateSelectionSlider(minimum, text):
    selectionSl.layout.display = 'block'
    selectionSl.layout.display = 'flex'
    selectionSl.layout.flex_flow = 'row'
    selectionSl.children[1].min = minimum
    selectionSl.children[0].value = text

def selectionBtnClicked(change):
    if change['new'] == 'najlepszych' or change['new'] == 'strategia elitarna':
        updateSelectionSlider(0, 'Procent osobników')
    if change['new'] == 'kołem ruletki':
        selectionSl.layout.display = 'none'
    if change['new'] == 'turniejowa':
        updateSelectionSlider(1, 'Wielkość turnieju')

def startApp(change):
    popSize = populationSl.children[1].value
    chromoBits = bitsSl.children[1].value
    epoch = epochSl.children[1].value
    selection = selectionBtn.value
    crossover = crossoverBtn.value
    mutation = mutationBtn.value
    if selection != 'kołem ruletki':
        selectionParam = selectionSl.children[1].value
    crossoverProb = crossoverSl.children[1].value
    mutationProb = mutationSl.children[1].value  
    
    app.children[0].layout.display = 'none'
    app.children[1].layout = Layout()
    epochProgress.max = epoch
    
    for i in range(epoch):
        time.sleep(0.1)
        incrementEpochProgress()

#### Panel konfiguracji parametrów

In [5]:
def initFirstPanel():
    global startBtn
    
    configLayout = initConfPanel()
    geneticLayout = initGenPanel()
    startBtn = initButton('Rozpocznij', 'success', '72%', '50px')
    startBtn.observe(startApp, 'value')
    
    return Box(
        children = [configLayout, geneticLayout, startBtn], 
        layout = Layout(display = 'flex', flex_flow = 'column'))

#### Panel progresu epok

In [6]:
def initProgressPanel():
    global epochProgress
    epochProgress = IntProgress(
        value = 0, min = 0, max = 10, step = 1, description = 'Epoka:', 
        bar_style = '', layout = Layout(display = 'none'))
    return epochProgress

#### Panel wyników: czas obliczeń, wykresy, zapis do pliku

In [7]:
def updateTime(value):
    timeOutput.children[1].value = value

def showPlot():
    pyplot.figure()
    pyplot.plot([0, 1, 2, 3, 4], [0, 3, 5, 9, 11])
    pyplot.xlabel('Months')
    pyplot.ylabel('Books Read')
    pyplot.show()

def showFirstPlot(value):
    if value['new'] == True:
        showPlot() # to change
    boothFuncBtn.value = False

def showSecondPlot(value):
    if value['new'] == True:
        showPlot() # to change
    meanBoothFuncBtn.value = False
    
def showThreePlot(value):
    if value['new'] == True:
        showPlot() # to change
    stdBtn.value = False

def restartApp(change):
    app.children[2].layout.display = 'none'
    app.children[0].layout =  Layout(display = 'flex', flex_flow = 'column')
    epochProgress.value = 0

def ifFinished(change):
    if change['new'] == epochProgress.max:
        app.children[1].layout.display = 'none'
        app.children[2].layout = Layout(display = 'flex', flex_flow = 'column')

In [8]:
def initResultPanel():
    global timeOutput, boothFuncBtn, meanBoothFuncBtn, stdBtn, fileInput, fileButton, returnBtn
    timeOutput = HBox([
        Label('Czas obliczeń: ', layout = Layout(width = '100px')), 
        Label('00.00 sek', layout = Layout(width = '100px'))])

    boothFuncBtn = initButton('Wykres zależności wartości funkcji celu od kolejnej iteracji', 'danger', '50%', '') 
    meanBoothFuncBtn = initButton('Wykres zależności średniej wartości funkcji celu od kolejnej iteracji', 'danger', '50%', '')
    stdBtn = initButton('Wykres zależności odchylenia standardowego od kolejnej iteracji', 'danger', '50%', '') 
    plotsBox = Box(children = [boothFuncBtn, meanBoothFuncBtn, stdBtn], 
        layout = Layout(display = 'flex', flex_flow = 'column', margin = "10px"))

    fileInput = HBox([Label('Zapis wyników do pliku:', layout = Layout(width = '200px')), 
        Text(value = '', placeholder = 'lokalizacja pliku', disabled = False, layout = Layout(width = '80%'))])
    fileButton = initButton('Zapisz', 'warning', '11%', '')
    fileBox = Box(children = [fileInput, fileButton], 
              layout = Layout(display = 'flex', flex_flow = 'row', margin = "10px"))

    returnBtn = initButton('Skonfiguruj ustawienia algorytmu ponownie', 'success', '50%', '')

    boothFuncBtn.observe(showFirstPlot, 'value')
    meanBoothFuncBtn.observe(showSecondPlot, 'value')
    stdBtn.observe(showThreePlot, 'value')
    returnBtn.observe(restartApp, 'value')
    epochProgress.observe(ifFinished, 'value')

    return Box(children = [timeOutput, plotsBox, fileBox, returnBtn], layout = Layout(display = 'none'))

In [9]:
def initApp(): 
    global app
    app = Box(
        children = [initFirstPanel(), initProgressPanel(), initResultPanel()], 
        layout = Layout(display = 'flex', flex_flow = 'column'))
    return app